In [1]:
import pandas as pd

pd.set_option('display.max_columns', 500)

def get_state_abbr():
    return {
        "AL": "Alabama",
        "AK": "Alaska",
        "AZ": "Arizona",
        "AR": "Arkansas",
        "CA": "California",
        "CO": "Colorado",
        "CT": "Connecticut",
        "DE": "Delaware",
        "FL": "Florida",
        "GA": "Georgia",
        "HI": "Hawaii",
        "ID": "Idaho",
        "IL": "Illinois",
        "IN": "Indiana",
        "IA": "Iowa",
        "KS": "Kansas",
        "KY": "Kentucky",
        "LA": "Louisiana",
        "ME": "Maine",
        "MD": "Maryland",
        "MA": "Massachusetts",
        "MI": "Michigan",
        "MN": "Minnesota",
        "MS": "Mississippi",
        "MO": "Missouri",
        "MT": "Montana",
        "NE": "Nebraska",
        "NV": "Nevada",
        "NH": "New Hampshire",
        "NJ": "New Jersey",
        "NM": "New Mexico",
        "NY": "New York",
        "NC": "North Carolina",
        "ND": "North Dakota",
        "OH": "Ohio",
        "OK": "Oklahoma",
        "OR": "Oregon",
        "PA": "Pennsylvania",
        "RI": "Rhode Island",
        "SC": "South Carolina",
        "SD": "South Dakota",
        "TN": "Tennessee",
        "TX": "Texas",
        "UT": "Utah",
        "VT": "Vermont",
        "VA": "Virginia",
        "WA": "Washington",
        "WV": "West Virginia",
        "WI": "Wisconsin",
        "WY": "Wyoming",
    }

In [4]:
solar = pd.read_csv('../data/projects/solar/solar_raw.csv')
solar.head()

,plant_code,plant_name,utility_na,utility_id,sector_nam,city,county,statename,zip,street_add,primsource,install_mw,total_mw,solar_mw,source_des,tech_desc,source,period,latitude,longitude,WKT,Unnamed: 21
0,10029,General Electric Aircraft Engines,General Electric Aircraft Engines,7049,Industrial Non-CHP,Lynn,Essex,Massachusetts,1910,1000 Western Avenue,solar,2.1,2.1,2.1,Solar = 2.1 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202108,42.450000,-70.973900,POINT Z (-70.9739 42.450000000000074 0),Massachusetts
1,10437,Sunray 2,"Sunray Energy 2, LLC",60246,IPP Non-CHP,Daggett,San Bernardino,California,92327,35100 Santa Fe Street,solar,20.0,20.0,20.0,Solar = 20 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202108,34.863356,-116.827627,POINT Z (-116.82762700000005 34.86335600000007...,California
2,10438,Sunray 3,Sunray Energy 3 LLC,60247,IPP Non-CHP,Daggett,San Bernardino,California,92327,35100 Santa Fe St,solar,13.8,13.8,13.8,Solar = 13.8 MW,Solar Photovoltaic,"EIA-860, EIA-860M and EIA-923",202108,34.863356,-116.827627,POINT Z (-116.82762700000005 34.86335600000007...,Arizona
3,10444,SEGS VIII,Terra-Gen Operating Co-Solar,61950,IPP Non-CHP,Hinkley,San Bernardino,California,92347,43880 Harper Lake Road,solar,92.0,80.0,80.0,Solar = 80 MW,Solar Thermal without Energy Storage,"EIA-860, EIA-860M and EIA-923",202108,35.030800,-117.356300,POINT Z (-117.35630000000009 35.03080000000018 0),Nevada
4,10446,SEGS IX,Terra-Gen Operating Co-Solar,61950,IPP Non-CHP,Hinkley,San Bernardino,California,92347,43880 Harper Lake Road,solar,92.0,88.0,88.0,Solar = 88 MW,Solar Thermal without Energy Storage,"EIA-860, EIA-860M and EIA-923",202108,35.033000,-117.338000,POINT Z (-117.33800000000005 35.0330000000001 0),Colorado


In [6]:
operating_years = solar['Operating Year'].unique()
solar_important = solar[['State', 'County', 'Operating Year', 'Nameplate Capacity (MW)']]
solar_important['State'] = solar_important['State'].map(get_state_abbr())

KeyError: 'Operating Year'

In [ ]:
dfs = {}
for operating_year in operating_years:
    grouped = solar_important[solar_important['Operating Year'] == operating_year].drop(columns=['Operating Year']).groupby(['State', 'County'])
    grouped_summed = grouped.sum().reset_index()
    grouped_mean = grouped.mean().reset_index()
    grouped_count = grouped.count().reset_index()
    
    merged = grouped_summed.merge(grouped_mean, on=['State', 'County'], suffixes=('_sum', '_mean')).merge(grouped_count, on=['State', 'County']).rename(columns={'Nameplate Capacity (MW)_sum': 'Nameplate Capacity (MW) sum', 'Nameplate Capacity (MW)_mean': 'Nameplate Capacity (MW) mean', 'Nameplate Capacity (MW)': 'Number of Plants'})
    
    dfs[operating_year] = merged

## Normalize

In [ ]:
# Get County Sizes
county_sizes = pd.read_csv('data/county_sizes.csv')
county_sizes.head()

In [ ]:
for year, df in dfs.items():
    merged = df.merge(county_sizes, left_on=['State', 'County'], right_on=['State', 'County'])
    merged['Nameplate Capacity (MW) sum - Norm mi2'] = merged['Nameplate Capacity (MW) sum'] / merged['area mi2']
    merged['Nameplate Capacity (MW) sum - Norm km2'] = merged['Nameplate Capacity (MW) mean'] / merged['area km2']
    
    merged['Number of Plants - Norm mi2'] = merged['Number of Plants'] / merged['area mi2']
    merged['Number of Plants - Norm km2'] = merged['Number of Plants'] / merged['area km2']
    
    dfs[year] = merged

In [ ]:
dfs[2019].head()